In [ ]:
# 引入需要的library
import numpy as np
import pandas as pd

In [ ]:
# 讀取海溫資料(2021~2025)

station = 'c6b01' # 彭佳嶼
# station = '1486' # 高雄
# 年份list
years = ['2021', '2022', '2023', '2024', '2025']
# dataframe存在list, 後面再合併
years_list = []
for year in years:
    # 彭佳嶼站第8行是標題，高雄站第9行是標題，skiprows要記得改
    df_temp = pd.read_csv(f'{station}_{year}.csv', skiprows=8)
    df_temp['date'] = df_temp['yyyymmddhhmi'].astype(str).str[:8]
    result = df_temp.groupby("date", as_index=False)['T'].mean()
    years_list.append(result)

all_temp = pd.concat(years_list, ignore_index=True)
print(all_temp.shape)


In [ ]:
# 自製函式：民國年月日轉西元年月日8碼
def roc_chinese_to_ad(date_str):
    # 去掉中文字
    date_str = date_str.replace("年", " ").replace("月", " ").replace("日", " ")
    parts = date_str.split()
    roc_year = int(parts[0])
    month = int(parts[1])
    day = int(parts[2])
    ad_year = roc_year + 1911
    return f"{ad_year:04d}{month:02d}{day:02d}"

# 港口變換
port_loc = '頭城'
# port_loc = '梓官'

# 讀取小卷行情資料，生產地分頁，僅讀取交易日期、交易資訊、頭城，去除空資料行；sp = squid price
sp = pd.read_excel('小卷行情資料.xlsx', sheet_name='生產地', usecols=['交易日期', '交易資訊', f'{port_loc}']).dropna()

# 轉換格式：把 "交易資訊" 轉成欄位(by ChatGPT)
sp_wide = sp.pivot(index="交易日期", columns="交易資訊", values=port_loc).reset_index()

# 注意：pivot 會自動排序 index，所以這裡用 Categorical 保留原始順序(by ChapGPT)
sp_wide["交易日期"] = pd.Categorical(sp_wide["交易日期"], categories=sp["交易日期"].unique(), ordered=True)
sp_wide = sp_wide.sort_values("交易日期").reset_index(drop=True)

# 轉換西元年月日8碼
sp_wide['交易日期'] = sp_wide['交易日期'].apply(roc_chinese_to_ad)

sp_wide.rename(columns={'交易日期': 'date'}, inplace=True)
print(sp_wide.shape)

In [ ]:
# 2021~2025海溫資料(僅彭佳嶼)合併，並去除遺失資料的日期 => all_temp, 有1260筆資料
# 2021~0225交易資料(僅頭城漁港)，並去除遺失資料的日期 => sp_wide(squid price in wide colomn), 有1149筆資料
# 兩個dataframe合併, 交集的有829筆資料(意即都有資料的天數有829天), 合併到merge_data, 後續可做分析作業

merge_data = pd.merge(all_temp, sp_wide, on='date', how='inner')
print(merge_data)

In [ ]:
# 依照海溫(T)、交易量(公斤)、平均價(元/公斤)，繪製3D回歸線
# by ChatGPT

# 載入必要Library
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D

# 多元回歸：溫度 + 交易量 → 價格
X = merge_data[["T", "交易量(公斤)"]]
y = merge_data["平均價(元/公斤)"]

model = LinearRegression()
model.fit(X, y)

print("回歸係數:", model.coef_)   # 對應 [temperature, volume]
print("截距:", model.intercept_)

# 預測
y_pred = model.predict(X)

# --- 繪圖 (3D) ---
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# 畫散點
ax.scatter(merge_data["T"], merge_data["交易量(公斤)"], y, color="blue", label="Data")

# 建立網格 (meshgrid)
x_surf, y_surf = np.meshgrid(
    np.linspace(merge_data["T"].min(), merge_data["T"].max(), 20),
    np.linspace(merge_data["交易量(公斤)"].min(), merge_data["交易量(公斤)"].max(), 20)
)
z_pred = model.predict(np.c_[x_surf.ravel(), y_surf.ravel()]).reshape(x_surf.shape)

# 畫回歸平面
ax.plot_surface(x_surf, y_surf, z_pred, color="red", alpha=0.5)

ax.set_xlabel("Temperature")
ax.set_ylabel("Volume")
ax.set_zlabel("Price")
ax.set_title("Multiple Regression: Temp + Volume → Price")

plt.show()


In [ ]:
# 比較炫砲可以滑鼠操控的3D圖, by ChatGPT

import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# 模型
X = merge_data[["T", "交易量(公斤)"]]
y = merge_data["平均價(元/公斤)"]
model = LinearRegression().fit(X, y)

# 建立格點
temp_range = np.linspace(merge_data["T"].min(), merge_data["T"].max(), 20)
vol_range = np.linspace(merge_data["交易量(公斤)"].min(), merge_data["交易量(公斤)"].max(), 20)
T, V = np.meshgrid(temp_range, vol_range)
P = model.predict(np.c_[T.ravel(), V.ravel()]).reshape(T.shape)

# 畫互動 3D
fig = go.Figure()

# 資料點
fig.add_trace(go.Scatter3d(
    x=merge_data["T"], y=merge_data["交易量(公斤)"], z=y,
    mode="markers", marker=dict(size=5, color="blue"),
    name="Data"
))

# 回歸平面
fig.add_trace(go.Surface(x=temp_range, y=vol_range, z=P, colorscale="Viridis", opacity=0.5))

fig.update_layout(
    scene=dict(
        xaxis_title="Temperature",
        yaxis_title="Volume",
        zaxis_title="Price"
    ),
    title="3D Regression Plane"
)

fig.show()


In [ ]:
# 3D回歸線不易看出變化
# 僅利用海溫(T)、平均價(元/公斤)，繪製2D回歸線，by ChatGPT

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# 建立模型
X = merge_data[["T"]]
y = merge_data["平均價(元/公斤)"]
model = LinearRegression().fit(X, y)

# 建立回歸線
temp_range = np.linspace(merge_data["T"].min(), merge_data["T"].max(), 100).reshape(-1, 1)
price_pred = model.predict(temp_range)

# 畫圖
plt.scatter(merge_data["T"], merge_data["平均價(元/公斤)"], color="blue", label="Data points")
plt.plot(temp_range, price_pred, color="red", label="Regression line")

plt.xlabel("Temperature")
plt.ylabel("Price")
plt.title("Temperature vs Price Regression")
plt.legend()
plt.grid(True)
plt.show()
